In [ ]:
# Import shapely objects
from shapely.geometry import Polygon, Point
from shapely.ops import unary_union

# Import numpy
import numpy as np

# Import scipy
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from scipy.interpolate import RectBivariateSpline as RBS

# Import skimage
from skimage.feature import peak_local_max
from skimage import measure

# import math tools
from math import radians, sin, cos, asin, sqrt

In [4]:
# function to compute local maxima
def locate_loc_maximum(TRA, threshold):
    xy = peak_local_max(TRA, threshold_abs = threshold)
    return xy

# function to check if point is inside closed curve (=vortex)
def check_point_inside_vortex(x, y, P):
    point = Point(x, y)
    return P.contains(point)

# Check if polygon object is inside polygon object
def check_polygon_inside_polygon(vertex_x, vertex_y, Poly_x, Poly_y, index):
    min_arg_x = np.argmin(vertex_x)
    max_arg_x = np.argmax(vertex_x)
    min_arg_y = np.argmin(vertex_y)
    max_arg_y = np.argmax(vertex_y)

    check_Poly = [[], [], [], []]

    for p in range(len(Poly_x)):
        points = np.array([Poly_x[p], Poly_y[p]]).T
        P = Polygon(points)
        check_Poly[0].append(int(check_point_inside_vortex(vertex_x[min_arg_x],
                                            vertex_y[min_arg_x], P)))
        check_Poly[1].append(int(check_point_inside_vortex(vertex_x[max_arg_x],
                                            vertex_y[max_arg_x], P)))
        check_Poly[2].append(int(check_point_inside_vortex(vertex_x[min_arg_y],
                                            vertex_y[min_arg_y], P)))
        check_Poly[3].append(int(check_point_inside_vortex(vertex_x[max_arg_y],
                                            vertex_y[max_arg_y], P)))

    if np.sum(check_Poly) > 0 or (len(check_Poly[0]) == 0):
        return True
    elif np.sum(check_Poly) == 0:
        return False
        
def gc_distance(lat1, lon1, lat2, lon2):

    # Earth radius in kilometers 
    R = 6378
 
    # Our formula requires we convert all degrees to radians
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    lon1 = radians(lon1)
    lon2 = radians(lon2)
 
    lat_span = lat1 - lat2
    lon_span = lon1 - lon2
 
    dist = 2 * R * asin(sqrt(sin(lat_span / 2) ** 2 + cos(lat1) * cos(lat2) * sin(lon_span / 2) ** 2))
 
    return dist      

In [1]:
def __extract_vortex__(X, Y, TRA, x_pos, y_pos, threshold_TRA_loc_max, N_drifters):
    
    # interpolant object for TRA-field
    interpolant_TRA = RBS(Y[:,0], X[0,:], TRA)
    
    # spacing of auxiliary grid for derivatives of gradient
    rho_x = 0.025*(X[0,1]-X[0,0])
    rho_y = 0.025*(Y[1,0]-Y[0,0])
    
    # resolution of contour increment
    res= 0.005
    
    # grid spacing
    dx = X[0,1]-X[0,0]
    dy = Y[1,0]-Y[0,0]
    
    # find local maxima in the TRA-field above a given threshold
    xy = locate_loc_maximum(TRA, threshold_TRA_loc_max)

    # Define list which stores the vertices of the vortex boundary
    vertex_x_hyper_lst, vertex_y_hyper_lst = [], []
    
    # Define list which stores location and TRA-value of local maxima
    x_max_lst, y_max_lst, TRA_local_maxima = [], [], []

    # associate local maxima to corresponding lists
    for idx_maxima in range(xy.shape[0]):
        x_max_lst.append(X[xy[idx_maxima, 0], xy[idx_maxima, 1]])
        y_max_lst.append(Y[xy[idx_maxima, 0], xy[idx_maxima, 1]])
        TRA_local_maxima.append(TRA[xy[idx_maxima, 0], xy[idx_maxima, 1]])
    
    convex_contour_x, convex_contour_y = [], []
    
    # iterate over all local maxima
    for idx_maxima in range(xy.shape[0]): 
        
        x_max = x_max_lst[idx_maxima]
        y_max = y_max_lst[idx_maxima]
        
        # Define candidate level set values which could be vortex boundaries
        levels = np.arange(0, TRA_local_maxima[idx_maxima]+res, res)
        
        vertex_x_lst, vertex_y_lst, grad_TRA_lst = [], [], []
        
        # Iterate through all the level sets and check if vortex criteria are satisfied
        for value in levels:

            # find contour to corresponding level set
            contours = measure.find_contours(TRA, value)
            
            # iterate over all contours with the given level set
            for vertices in contours:
                
                # long, lat coordinates of contour
                x_path = np.min(X)+vertices[:, 1]*dx
                y_path = np.min(Y)+vertices[:, 0]*dy
                
                if len(x_path) > 4:
                
                    # transfrom vertices to points
                    points = np.array([x_path, y_path]).T
                    
                    # check if contour is closed
                    if points[0, 0] == points[-1, 0] and points[0, 1] == points[-1, 1]:
                        
                        # compute Polygon object out of points
                        Poly = Polygon(points)
                
                         # Check if eddy candidate contains a local maximum
                        check_bool_local_max = check_point_inside_vortex(x_max, y_max, Poly)
                        
                        if check_bool_local_max:
                            
                            # check if closed curve contains drifters
                            check_bool_in = []
                            for idx_x in range(len(x_pos)):
                                check_bool_in.append(check_point_inside_vortex(x_pos[idx_x], y_pos[idx_x], Poly))
                            
                            # if number of drifters inside the closed curve is greater than N_drifters
                            # then vortex satisfies all the criteria
                            check_bool = (np.sum(check_bool_in) >= N_drifters)
                            if np.sum(check_bool_in) >= N_drifters:
                                
                                vertex_x = points[:, 0]
                                vertex_y = points[:, 1]
                                vertex_x_lst.append(vertex_x)
                                vertex_y_lst.append(vertex_y) 
                                
                                x0 = vertex_x.ravel()
                                y0 = vertex_y.ravel()
                                
                                xR = x0+rho_x
                                xL = x0-rho_x
                                
                                yU = y0+rho_y
                                yD = y0-rho_y
                                
                                dTRAdx = (interpolant_TRA(y0, xR, grid = False)-interpolant_TRA(y0, xL, grid = False))/(2*rho_x)
                                dTRAdy = (interpolant_TRA(yU, x0, grid = False)-interpolant_TRA(yD, x0, grid = False))/(2*rho_y)
                                
                                grad_TRA_avg = np.mean(np.sqrt(dTRAdx**2+dTRAdy**2))
                                
                                grad_TRA_lst.append(grad_TRA_avg)
        
        if len(grad_TRA_lst) > 0:
            
            # sort closed curves from maximum to minimum area 
            grad_TRA_sorted, vertex_x_sorted = zip(*sorted(zip(grad_TRA_lst, vertex_x_lst), reverse = True))
            grad_TRA_sorted, vertex_y_sorted = zip(*sorted(zip(grad_TRA_lst, vertex_y_lst), reverse = True))
        
            points = np.array([vertex_x_sorted[0], vertex_y_sorted[0]]).transpose()
            
            if points.shape[0] > 4:
        
                hull = ConvexHull(points)
                
                convex_contour_x.append(np.append(points[hull.vertices, 0], points[hull.vertices[0], 0]))
                convex_contour_y.append(np.append(points[hull.vertices, 1], points[hull.vertices[0], 1]))
    
    # if two convex curves intersect --> unite them and generate a new convex curve 
    # given by the convex hull of the two intersecting curves
    
    Poly_objects = []
    
    for i in range(len(convex_contour_x)):
        Poly_objects.append(Polygon(np.array([convex_contour_x[i].ravel(), convex_contour_y[i].ravel()]).T))
    
    coord_boundary_x, coord_boundary_y, centroid = [], [], []
    
    Multi_poly = unary_union(Poly_objects)
    
    if np.str(type(Multi_poly)) == "<class 'shapely.geometry.polygon.Polygon'>":
        
        x = np.asarray(Multi_poly.exterior.coords)[:,0]
        y = np.asarray(Multi_poly.exterior.coords)[:,1]
        
        points = np.array([x, y]).transpose()
        
        hull = ConvexHull(points)
        
        coord_boundary_x.append(np.append(points[hull.vertices, 0], points[hull.vertices[0], 0]))
        coord_boundary_y.append(np.append(points[hull.vertices, 1], points[hull.vertices[0], 1]))
        
        # Calculate geometric centroid of convex hull   
        centroid.append(np.mean(points[hull.vertices, :], axis=0))
        
        return coord_boundary_x, coord_boundary_y, centroid
    
    for poly in Multi_poly:
        
        # convex hull of  poly object
        x = np.asarray(poly.exterior.coords)[:,0]
        y = np.asarray(poly.exterior.coords)[:,1]
        
        points = np.array([x, y]).transpose()
        
        hull = ConvexHull(points)
        
        x_hull = np.append(points[hull.vertices, 0], points[hull.vertices[0], 0])
        y_hull = np.append(points[hull.vertices, 1], points[hull.vertices[0], 1])
        
        coord_boundary_x.append(x_hull)
        coord_boundary_y.append(y_hull)
            
        # Calculate geometric centroid of convex hull   
        centroid.append(np.mean(points[hull.vertices, :], axis=0))
                        
    return coord_boundary_x, coord_boundary_y, centroid